# Detecting outlyers with minimal volume ellipse





In [2]:
plot_inline = False
if plot_inline:
    %matplotlib inline

# Future 
from __future__ import print_function

# Numpy imports 
import numpy as np
import numpy.random as random

import scipy as sp
import scipy.linalg as la



# Convex optimization package: CVXOPT

## Import the basic packages
from cvxopt import matrix, solvers
#solvers.options['show_progress'] = False

## Import the CVXOPT version of LAPACK
from cvxopt import blas, lapack, sqrt, mul, cos, sin, log

# Import matplotlib.pyplot
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


EPS = sp.finfo(float).eps


# 2D scatter plot example 

In [3]:
# Generate random samples for testing the algorithm 
mean = np.array([5.0, 5.0, 5.0])
cov = np.array([[3.0, 0.5, 0.8], 
                [0.5, 1.1, 0.8], 
                [0.8, 0.8, 1.0]])
Nsamples = 10000
samples = random.multivariate_normal(mean, cov, Nsamples)
samples[:50] = random.laplace(5, scale =.0000001, size=(50, mean.size))
#plt.plot(samples[:, 0], samples[:, 1], 'or', markersize=4)
#plt.axis('equal')


# Make a 3D figure
if plot_inline:
    fig = plt.figure(0)
    fig.clear()
    ax = fig.add_subplot(1,1,1, projection='3d')
    ax.scatter(samples[:,0], 
               samples[:,1],
               samples[:,2], marker='.')




In [4]:
# Define function for various array calculations


## What is the dimensionality 
ndim = samples.shape[-1]

## ellipsoid free parameters
Na = ndim + ndim * (ndim + 1) / 2.0


## Parameterization of the ellipse as a single vector
def get_DB(ndim, Na):
    """
    Define the array Dnij so that, 
      Aij = SUM_n Dnij * an. 
      
    Define the array Bni so that, 
      bi = SUM_n Bni * an.
    """
    # Zero array of correct size
    D = sp.zeros((Na, ndim, ndim))

    # Loop over parameters for ellipse
    n=0
    for i in range(ndim):
        for j in range(0, i+1):
                        
            D[n, i, j] = 1.0
            D[n, j, i] = 1.0

            n+=1

    # initialize B
    B = sp.zeros((Na, ndim))
    B[-ndim:, :] = sp.eye(ndim)
        
    return D, B
    
    
D, B = get_DB(ndim, Na)
    
    
 
print(D, end='\n\n')
print(B)

[[[ 1.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  1.  0.]
  [ 1.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  0.]
  [ 0.  1.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  1.]
  [ 0.  0.  0.]
  [ 1.  0.  0.]]

 [[ 0.  0.  0.]
  [ 0.  0.  1.]
  [ 0.  1.  0.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  1.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]]

[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]


/home/willm/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:21: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/willm/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:34: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [5]:
#D = sp.zeros((5, 2, 2), dtype=float)
#D[2:] = sp.array([[[1, 0], 
#                   [0, 0]], 
#                  [[0, 1], 
#                   [1, 0]], 
#                  [[0, 0], 
#                   [0, 1]]])
    
#B = sp.zeros((5, 2), dtype=float)
#B[:2] = sp.array([[1, 0], [0, 1]])
#print(D, end='\n\n')
#print(B)
from scipy import special

In [6]:

## Functions
def get_A(a):
    return sp.tensordot(D, a, axes=[(0,), (0,)])
def get_b(a):
    return sp.tensordot(B, a, axes=[(0,), (0,)])

## Cost function 
def get_f0(a):
    "Compute the objective function: -log(det(A(a)))."
    A = get_A(a)
    #print(A)
    Ainv = la.inv(A)
    out = sp.log(la.det(Ainv))
    return out #-1.0 * sp.log(la.det(A))
    
def grad_f0(a, D=D):
    "Compute the gradient of the objective function: -log(det(A(a)))."
    A = get_A(a)
    Ainv = la.inv(A)
    E = sp.dot(Ainv, D).transpose(1,0,2)
    out = -sp.trace(E, axis1=1, axis2=2)
    return out

def hess_f0(a, D=D):
    "Compute the hessian of the objective function: -log(det(A(a)))."
    A = get_A(a)
    Ainv = la.inv(A)
    E = sp.dot(Ainv, D).transpose(1,0,2)
    EE = sp.dot(E, E).trace(axis1=1, axis2=3)
    H = (1.0 / 2.0) * (EE + EE.T)
    return H
     

In [7]:
## Check the derivative calculations    

a0 = sp.array([.10, 0.005, 0.1, 0.005, 0.005, .1, .5, .5, 0.5])


msg = "The objective value is f0: \n\t{0}\nThe gradient grad(f0) is :\n\t{1}\n\n"
print(msg.format(get_f0(a0), grad_f0(a0)))

df0 = grad_f0(a0)
h = 1e-7 * df0

grad_diff = (get_f0(a0 + h) - get_f0(a0 - h))/(2 * la.norm(h))
grad_func = la.norm(df0)
hess_diff = (get_f0(a0 + h) - 2.0 * get_f0(a0) + get_f0(a0 - h)) / (la.norm(h))**2
hess_func = sp.dot(h, sp.dot(hess_f0(a0), h)) / (la.norm(h)**2)


msg = "Gradient error :   {0}"
print(msg.format(abs(grad_diff - grad_func) / grad_diff))
msg = "Hessian error  :   {0}"
print(msg.format(abs(hess_diff - hess_func) / hess_diff))

The objective value is f0: 
	6.91503168795
The gradient grad(f0) is :
	[-10.04784689   0.9569378  -10.04784689   0.9569378    0.9569378
 -10.04784689  -0.          -0.          -0.        ]


Gradient error :   3.90813887179e-11
Hessian error  :   1.03081178615e-07


In [8]:
# Define the constraint function

## The constraint function 
def get_f1(a, x=samples, B=B, D=D):
    "Define the nth constraint function."
    b = get_b(a)
    A = get_A(a)
    Ax = sp.dot(A, x.T)
    val = ((b[:, None] + Ax[:, :])**2).sum(0) - 1.0
    return val

def grad_f1(a, x=samples, B=B, D=D):
    "Define the gradient for each convex inequality."
    b = get_b(a)
    A = get_A(a)
    Ax = sp.dot(A, x.T)
    vec0 = b[:, None] + Ax[:, :]                      #  in
    vec1 = B[:, :, None] + sp.dot(D, x.T)[:, :, :]    # kin
    vec_isum = 2.0 * (vec0[None, :, :] * vec1[:, :, :]).sum(1).transpose()
    return vec_isum

def hess_f1(a, x=samples, B=B, D=D):
    "Define the hessians for each convex inequality."
    vec1 = B[:, :, None] + sp.dot(D, x.T)[:, :, :]    # kin -> nkk'
    vec_kkn = 2.0 * (vec1[:, None, :, :] * vec1[None, :, :, :]).sum(2)
    vec_nkk = vec_kkn.transpose(2,0,1)
    return vec_nkk

In [9]:
## Check the derivative calculations    
#a0 = sp.array([.10, .10, .050, 0.005, .050])

a0 = sp.array([.05, 0.005, 0.05, 0.005, 0.005, 0.05, 0.1, 0.1, 0.1])

msg = "The objective value is f1: \n\t{0}\nThe gradient grad(f0) is :\n\t{1}\n\n"
print(msg.format(get_f0(a0), grad_f0(a0)))

f10 = lambda a: get_f1(a)[0]
grad_f10 = lambda a: grad_f1(a)[0]
hess_f10 = lambda a: hess_f1(a)[0]

df0 = grad_f10(a0)
h = 1e-7 * df0

grad_diff = (f10(a0 + h) - f10(a0 - h))/(2 * la.norm(h))
grad_func = la.norm(df0)
hess_diff = (f10(a0 + h) - 2.0 * f10(a0) + f10(a0 - h)) / (la.norm(h))**2
hess_func = sp.dot(h, sp.dot(hess_f10(a0), h)) / (la.norm(h)**2)


msg = "Gradient error :   {0}"
print(msg.format(abs(grad_diff - grad_func) / grad_diff))
msg = "Hessian error  :   {0}"
print(msg.format(abs(hess_diff - hess_func) / hess_diff))

The objective value is f1: 
	9.01559629518
The gradient grad(f0) is :
	[-20.37037037   3.7037037  -20.37037037   3.7037037    3.7037037
 -20.37037037  -0.          -0.          -0.        ]


Gradient error :   1.36957537793e-12
Hessian error  :   2.24368710471e-07


## Define the interface function for CVXOPT

In [10]:
# Loewner-John ellipsoid
#
# minimize     log det A^-1
# subject to   || A(a) x_n + b(a) ||_2^2  - 1.0 <= 0  for  n=1,...,m
#
# 5 variables a = [b0, b1, A00, A01, A11]

def F(a=None, z=None, a0=a0, x=samples, B=B, D=D):
    
    # Convert input to numpy arrays
    n = a0.size
    m = x.shape[0]
    
    ## Handle the case for determining the initial value
    if a is not None:
        a = sp.array(a).reshape(n)
    else:
        return m, matrix(a0)
    
    
    ## Calculations 
    if z is not None: 
        z = sp.array(z).reshape(m + 1)
        
        ## Compute the full outputHessian
        fout = sp.zeros([m+1])
        dfout = sp.zeros([m+1, n])
        ddfout = sp.zeros([n, n])

        fout[0] = get_f0(a)
        dfout[0] = grad_f0(a)
        ddfout[:, :] += z[0] * hess_f0(a)
        
        fout[1:] = get_f1(a)
        dfout[1:, :] = grad_f1(a)
        ddfout[:, :] += (z[1:, None, None] * hess_f1(a)).sum(0)
         
        # Return the full output
        return (matrix(fout.reshape(m+1, 1)), 
                matrix(dfout), 
                matrix(ddfout))
    
    else:
        
        # Check the domain
        A = get_A(a)
        lams = la.eigvalsh(A)
        if lams.min() <= -10 * EPS:
            return (None, None)
        
        ## Compute partial output without hessian
        fout = sp.zeros([m+1])
        dfout = sp.zeros([m+1, n])

        fout[0] = get_f0(a)
        dfout[0] = grad_f0(a)
        
        fout[1:] = get_f1(a)
        dfout[1:, :] = grad_f1(a)

        return (matrix(fout.reshape(m+1, 1)), 
                matrix(dfout))
  


## Solve for the minimum volume ellipsoid containing all the points


In [11]:
sol = solvers.cp(F)
solx = np.array(sol['x']).flatten()
#print(solx)
b = get_b(solx)
A = get_A(solx)

Ainv = la.inv(A)

print(A.shape)
print(Ainv.shape)
print(b.shape)

     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -5.0335e+03  1e+04  1e+00  1e+00
 1:  9.8297e+00 -1.7788e+03  2e+03  8e-02  2e-01
 2:  9.7613e+00 -5.6505e+01  7e+01  2e-03  8e-03
 3:  7.8460e+00  2.3987e+00  6e+00  1e-02  7e-04
 4:  5.8845e+00  5.8636e+00  7e-01  3e-02  9e-05
 5:  5.2639e+00  5.2722e+00  6e-01  5e-02  7e-05
 6:  5.0320e+00  5.0070e+00  6e-01  5e-02  6e-05
 7:  3.9461e+00  4.1088e+00  4e-01  2e-01  4e-05
 8:  3.9335e+00  4.0918e+00  4e-01  2e-01  4e-05
 9:  3.8894e+00  4.0322e+00  4e-01  2e-01  4e-05
10:  3.8879e+00  4.0298e+00  4e-01  2e-01  4e-05
11:  3.8826e+00  4.0207e+00  4e-01  2e-01  4e-05
12:  7.3556e+00  3.1556e+00  5e+00  1e-02  5e-04
13:  7.2027e+00  3.0671e+00  5e+00  1e-02  5e-04
14:  7.0506e+00  2.9722e+00  4e+00  1e-02  5e-04
15:  6.9754e+00  2.9202e+00  4e+00  1e-02  5e-04
16:  6.9008e+00  2.8672e+00  4e+00  1e-02  4e-04
17:  6.8292e+00  2.8150e+00  4e+00  1e-02  4e-04
18:  6.7110e+00  2.7683e+00  4e+00  1e-02  4e-04
19:  6.6042e+00  2.71

# Make a plot showing min. vol ellipsoid

In [12]:
## Get points on a sphere
thetas = sp.linspace(100*EPS, (1.0-100*EPS) * sp.pi, 20)
phis = sp.linspace(0.0, 2*sp.pi, 20)

TT, PP = sp.meshgrid(thetas, phis, indexing='ij')

xx = sp.cos(PP) * sp.sin(TT)
yy = sp.sin(PP) * sp.sin(TT)
zz = sp.cos(TT)


Ainv_xx = Ainv[:,0, None, None] * (xx - b[None, None, 0])
Ainv_yy = Ainv[:,1, None, None] * (yy - b[None, None, 1])
Ainv_zz = Ainv[:,2, None, None] * (zz - b[None, None, 2])

xyz_ellipse = Ainv_xx + Ainv_yy + Ainv_zz
xe = xyz_ellipse[0]
ye = xyz_ellipse[1]
ze = xyz_ellipse[2]

print(Ainv_xx.shape)




(3, 20, 20)


# Determine candidats for removal

In [13]:
def xinxout(x, A=A, b=b):
    "Define the signed distance to the boundary."
    v = sp.dot(A, x.T) + b[:, None]
    Iin = la.norm(v, axis=0) <  (1 - 100 * sp.sqrt(EPS))
    Iout = la.norm(v, axis=0) >=  (1 - 100 * sp.sqrt(EPS))

    xin = x[Iin, :]
    xout = x[Iout, :]
    return xin, xout

xin, xout = xinxout(samples)

In [16]:

fig = plt.figure(1, figsize=(8,8))
fig.clear()
ax = fig.add_subplot(1,1,1,projection='3d')

ax.plot_wireframe(xe, ye, ze, linewidth=0.1)


ax.scatter(xin[:,0], xin[:,1], xin[:,2], c='b', marker='.', linewidth=0.1)
ax.scatter(xout[:,0], xout[:,1], xout[:,2], marker='o', c='r')



fig.show()

In [17]:
#plt.show()